In [ ]:
import sys
import os
import glob
from skimage import io
from matplotlib import pyplot as plt
from skimage.transform import resize, rotate
from skimage import color
from skimage.morphology import closing, opening, disk, square
from skimage.transform import probabilistic_hough_line
from skimage.feature import canny
import math 
import numpy as np
from skimage.feature import corner_harris, corner_subpix, corner_peaks
from skimage.filters import threshold_otsu
from skimage.util import img_as_bool, img_as_ubyte

# tymczasowe importy do testów
from skimage.measure import find_contours, approximate_polygon, regionprops, label
from skimage.color import label2rgb
from skimage.draw import circle_perimeter, rectangle_perimeter, circle, line
from matplotlib import cm

In [ ]:
# util
def get_line_length(point1, point2):
    return math.pow((point1[0]-point2[0])**2 + (point1[1]-point2[1])**2 , 0.5)

def get_angle_of_line(line):
    (point1, point2) = line
    angle = math.atan2(point2[1] - point1[1], point2[0] - point1[0]) * 180 / np.pi
    return angle #if angle >= 0 else 180+angle

def compare_2_numbers_with_range(n1, n2, range=3):
    difference = abs(n1-n2)
    return difference <= range

def compare_2_points_with_range(point1, point2, range=3):
    x1, y1 = point1
    x2, y2 = point2
    return compare_2_numbers_with_range(x1, x2, range) and compare_2_numbers_with_range(y1, y2, range)

def clamp(n, smallest, largest): 
    return max(smallest, min(n, largest))

def get_white_to_black_ratio_in_array(pixels_array):
    white_pixel_count = 0
    black_pixel_count = 0
    
    unique_values, counts = np.unique(pixels_array, return_counts=True)
    unique_counts_dict = dict(zip(unique_values, counts))
    
    if False in unique_counts_dict:
        black_pixel_count = unique_counts_dict[False]
    if True in unique_counts_dict:
        white_pixel_count = unique_counts_dict[True]
    

    ratio = white_pixel_count/(white_pixel_count + black_pixel_count)
    return ratio

In [ ]:
# image operation
def slice_image_in_radius(image, point, radius):
    (height, width) = point
    height = int(height)
    width = int(width)
    (image_height, image_width) = image.shape
    

    point_height_in_sliced_image = radius+1
    point_width_in_sliced_image = radius+1
    if height-radius < 0 :
        point_height_in_sliced_image = radius - (radius-height)
    if width-radius < 0:
        point_width_in_sliced_image = radius - (radius-width)
    middle_point = (point_height_in_sliced_image, point_width_in_sliced_image)

    right_width_min = clamp(width-radius-1, 0, image_width)
    right_width_max = clamp(width+radius+1, 0, image_width)
    right_height_min = clamp(height-radius-1, 0, image_height)
    right_height_max = clamp(height+radius+1, 0, image_height)
    slice_of_image = image[right_height_min:right_height_max, right_width_min:right_width_max]
    return slice_of_image, middle_point

def rotate_image(image, angle, baseline):
    baseline = [(el[1],el[0]) for el in baseline]
    is_baseline_top = True
    is_baseline_left = True
    
    (image_height, image_width) = image.shape
    (base_point1, base_point2) = baseline
    
    baseline_avg_height = (base_point1[0]+base_point2[0])/2
    baseline_avg_width = (base_point1[1]+base_point2[1])/2
    if baseline_avg_height <= image_height/2:
        is_baseline_top = True
    else:
        is_baseline_top = False
        
    if baseline_avg_width <= image_width/2:
        is_baseline_left = True
    else:
        is_baseline_left = False
        
    if angle >= 0:
        if not is_baseline_left:
            return img_as_bool(rotate(image, 180+angle, resize=True))
        else:
            return img_as_bool(rotate(image, angle, resize=True))

    if angle < 0:
        if is_baseline_left:
            return img_as_bool(rotate(image, 180+angle, resize=True))
        else:
            return img_as_bool(rotate(image, angle, resize=True))
    

def max_height_and_width_from_images(images):
    max_height, max_width = images[0].shape
    for image in images:
        if image.shape[0] > max_height:
            max_height = image.shape[0]
        if image.shape[1] > max_width:
            max_width = image.shape[1]
    return max_height, max_width

def min_height_and_width_from_images(images):
    min_height, min_width = images[0].shape
    for image in images:
        if image.shape[0] < min_height:
            min_height = image.shape[0]
        if image.shape[1] < min_width:
            min_width = image.shape[1]
    return min_height, min_width

def resize_image(image, max_height, max_width):  
    scale = max_width/image.shape[1]
    new_shape = (int(image.shape[0]*scale), int(image.shape[1]*scale))
    image_resized = (resize(img_as_ubyte(image), new_shape))
    image_resized = threshold_image(image_resized)
    
    return image_resized



def cut_image_sides(image):
    label_img, num = label(image, connectivity=2, return_num=True, background=0)
    region_props = regionprops(label_img)
    biggest_region = region_props[0]
    for region in region_props:
        if region.area > biggest_region.area:
            biggest_region = region
    image_cut = image[biggest_region.bbox[0]-2:biggest_region.bbox[2],biggest_region.bbox[1]:biggest_region.bbox[3]+1]
    return image_cut

def threshold_image(image):
    thresh = threshold_otsu(image)
    thresholded = image > thresh
#     cleared = clear_border(thresholded)
    return thresholded

def open_close_image(image):
    image = opening(closing(image, disk(2)), disk(2))
    


#     # jeżeli wyższy niż szerszy to obróć
#     if image.shape[1] < image.shape[0]:
#         image = img_as_bool(rotate(image, 90, resize=True))
        
#     first_row = image[0, :]
#     unique_values, counts = np.unique(first_row, return_counts=True)
#     unique_counts_dict = dict(zip(unique_values, counts))
#     white_pixel_count = 0
#     black_pixel_count = 0
        
#     # dodatkowe sprawdzenie czy nie jest do góry nogami
#     if False in unique_counts_dict:
#         black_pixel_count = unique_counts_dict[False]
#     if True in unique_counts_dict:
#         white_pixel_count = unique_counts_dict[True]
    
# #     print(white_pixel_count/(white_pixel_count + black_pixel_count))
# #     print("==============================")
#     if white_pixel_count/(white_pixel_count + black_pixel_count) > 0.8:
#         image = img_as_bool(rotate(image, 180, resize=True))
    return image

def add_black_border_to_image(image, width_of_border=30):
    (h, w) = image.shape
    new_image_left_border = np.zeros((h,w+width_of_border))
    new_image_left_border[:,width_of_border:] = image
    image = new_image_left_border
    
    (h, w) = image.shape
    new_image_right_border = np.zeros((h,w+width_of_border))
    new_image_right_border[:,:-width_of_border] = image
    image = new_image_right_border
    
    (h, w) = image.shape
    new_image_bot_border = np.zeros((h+width_of_border,w))
    new_image_bot_border[:-width_of_border,:] = image
    image = new_image_bot_border
    
    (h, w) = image.shape
    new_image_top_border = np.zeros((h+width_of_border,w))
    new_image_top_border[width_of_border:,:] = image
    image = new_image_top_border
    return image

def rotate_image_if_wrong_orientation(image):
    # dodatkowe sprawdzenie czy nie jest do góry nogami
    white_black_ratio_first_row = get_white_to_black_ratio_in_array(image[0, :])
    
    if white_black_ratio_first_row > 0.8:
        image = img_as_bool(rotate(image, 180, resize=True))
        

    # sprawdzenie czy nie wykryło podstawy jako lewego boku, czyli czy nie leży na lewym boku 
    white_black_ratio_left_column = get_white_to_black_ratio_in_array(image[:, 0])
    white_black_ratio_right_column = get_white_to_black_ratio_in_array(image[:, -1]) 

    if (white_black_ratio_right_column > 0.9 and white_black_ratio_left_column < 0.1):
        image = img_as_bool(rotate(image, -90, resize=True))
        
    # sprawdzenie czy nie wykryło podstawy jako prawego boku, czyli czy nie leży na prawym boku 
    white_black_ratio_left_column = get_white_to_black_ratio_in_array(image[:, 0])
    white_black_ratio_right_column = get_white_to_black_ratio_in_array(image[:, -1]) 

    if (white_black_ratio_right_column < 0.1 and white_black_ratio_left_column > 0.9) :
        image = img_as_bool(rotate(image, 90, resize=True))
    
    return image

In [ ]:
# v1
# wersja używa najdłuższej linii w figurze jako podstawy, nie zawsze będzie działało dla późniejszych przykałdów
def rotate_scale_and_cut_image_v1(image, image_path, max_height, max_width):
    image = threshold_image(image)
    angle, baseline = get_angle_to_rotate_image_based_on_longest_line(image)
    image = rotate_image(image, angle, baseline)
    image = cut_image_sides(image)
    image = resize_image(image, max_height, max_width)
    image = open_close_image(image)
    image = rotate_image_if_wrong_orientation(image)
    return image

def get_angle_to_rotate_image_based_on_longest_line(image):
    thresh = threshold_otsu(image)
    image_binary = image > thresh
    edges = canny(image_binary, sigma=1, low_threshold=False, high_threshold=True)
#     min_length = image.shape[0] if image.shape[0] >= image.shape[1] else image.shape[1]
    all_detected_lines = list()
    lines = probabilistic_hough_line(edges, threshold=10, line_length=30,line_gap=10)
    all_detected_lines += lines
    counter_of_tries = 0
    while counter_of_tries < 3:
        edges = canny(image_binary, sigma=1, low_threshold=False, high_threshold=True)
        min_length = image.shape[0] if image.shape[0] >= image.shape[1] else image.shape[1]
        lines = probabilistic_hough_line(edges, threshold=10, line_length=30,line_gap=10)
        all_detected_lines += lines
        counter_of_tries += 1

    longest_line = (lines[0],  get_line_length(lines[0][0], lines[0][1]))
    for line in lines:
        temp_lenght = get_line_length(line[0], line[1])
        if temp_lenght > longest_line[1]:
            longest_line = (line, temp_lenght)

    angle = get_angle_of_line(longest_line[0])
    baseline = longest_line[0]
    return angle, baseline

In [ ]:
# v3
# wersja wycina kawałki na około wykrytych punktów i na tej podstawie szuka katów prostych i podstawy
def rotate_scale_and_cut_image_v3(image, image_path, max_height, max_width):
    image = add_black_border_to_image(image, width_of_border=30)
    image = threshold_image(image)
    points_using_angles_of_lines = get_points_with_right_angle_using_angles_of_lines(image)
    points_using_areas_of_neighborhood = get_points_with_right_angle_using_areas_of_neighborhood(image)
    
    baseline = list(points_using_angles_of_lines.intersection(points_using_areas_of_neighborhood))
    baseline = [(el[1],el[0]) for el in baseline]
    if len(baseline) != 2:
        baseline = None
    if baseline == None:
        print('Nie znaleziono podstawy dla obrazka z użyciem v3: {0}'.format(image_path))
        return rotate_scale_and_cut_image_v1(image, image_path, max_height, max_width) #todo

    angle = get_angle_of_line(baseline)
    image = rotate_image(image, angle, baseline) 
    image = cut_image_sides(image)
    image = resize_image(image, max_height, max_width)
    image = open_close_image(image)
    return image

def get_points_with_right_angle_using_angles_of_lines(image):
    possible_baseline_points = corner_peaks(corner_harris(image), min_distance=10)
    points_with_right_angle = set()
#     print("possible_baseline_points: ", possible_baseline_points)
    for c in possible_baseline_points:
        if check_if_it_is_right_angle_using_hough(image=image, point=(c[0],c[1]), radius=30):
            points_with_right_angle.add((c[0],c[1]))
    
    return points_with_right_angle

def check_if_perpendicular_lines(line1, line2, accuracy_in_degrees=10):
    point11, point12 = line1
    point21, point22 = line2
#     print("line1: ",line1 )
#     print("line2: ",line2 )
    if compare_2_points_with_range(point11, point21, 10) or compare_2_points_with_range(point11, point22, 10) or compare_2_points_with_range(point12, point21, 10):     
        angle_between_lines = abs(get_angle_of_line(line1)-get_angle_of_line(line2))
        print(angle_between_lines)
        if compare_2_numbers_with_range(angle_between_lines, 90, accuracy_in_degrees) or compare_2_numbers_with_range(angle_between_lines, 270, accuracy_in_degrees):
            return True
    return False

# def check_if_parallel_lines(line1, line2, accuracy_in_degrees=10):
#     point11, point12 = line1
#     point21, point22 = line2
#     if compare_2_points_with_range(point11, point21, 10) or compare_2_points_with_range(point11, point22, 10) or compare_2_points_with_range(point12, point21, 10):     
#         angle_between_lines = abs(get_angle_of_line(line1)-get_angle_of_line(line2))
#         if compare_2_numbers_with_range(angle_between_lines, 0, accuracy_in_degrees) or compare_2_numbers_with_range(angle_between_lines, 360, accuracy_in_degrees):
#             return True  
#     return False

def get_list_of_different_angles_in_image(image, positions_of_white_pixels, radius, middle_point):
    list_of_angles = list()
    image = color.gray2rgb(image)
    image[middle_point] = (255,0,0)
    for white_point in positions_of_white_pixels:
        add = True
        if get_line_length(middle_point, white_point) > 10:
            point_angle = get_angle_of_line((middle_point, white_point))
            for angle in list_of_angles:
                if compare_2_numbers_with_range(point_angle, angle, 20):
                    add = False
            if add:
                image[white_point] = (255,0,0)
                list_of_angles.append(point_angle)
                
    # np. kąty -176 i 180 są obok siebie, zależą od zwrotu, to samo z kątem 90
    # tutaj usuwamy takie przypadki, żeby mieć na penwo różne kąty
    exists_90 = False
    exists_180 = False
    modified_list_of_angles = list()
    for angle in list_of_angles:
        if compare_2_numbers_with_range(90, abs(angle), 10):
            if exists_90 == False:
                exists_90 = True
                modified_list_of_angles.append(angle)
        elif compare_2_numbers_with_range(180, abs(angle), 10):
            if exists_180 == False:
                exists_180 = True
                modified_list_of_angles.append(angle)
        else:
            modified_list_of_angles.append(angle)
        
    return modified_list_of_angles, image

def check_if_it_is_right_angle_using_angles_of_lines(image, point, radius=10):
    precision_of_angles_of_points = 3
    slice_of_image, middle_point = slice_image_in_radius(image, point, radius)
    
    edges = canny(slice_of_image)
    positions_of_white_pixels = np.where(edges==True)
    positions_of_white_pixels = list(zip(positions_of_white_pixels[0],positions_of_white_pixels[1]))
    
    list_of_angles, slice_of_image = get_list_of_different_angles_in_image(slice_of_image, positions_of_white_pixels, radius, middle_point)
#     print("list_of_angles: ", list_of_angles)
    # jeżeli mamy tylko 1 kąt na wycinku obrazu to powinien składać się z 2 linii, 
    # te 2 linie powinny być pod zupełnie innymi kątami (różnią się o 90 stopni jeżeli prosty)
    # dlatego musimy mieć tylko 2 różne kąty pod jakimi przebiegają te dwie linie
    if len(list_of_angles) == 2:
        n_of_pixels = 3 # liczba pixeli, której szukamy aby zidentyfikować jedną linię
        
        first_list_points_in_one_line = list()
        for white_point in positions_of_white_pixels:
            if get_line_length(middle_point, white_point) > radius/2:
                angle_of_line = get_angle_of_line((middle_point, white_point))
                if compare_2_numbers_with_range(angle_of_line, list_of_angles[0], precision_of_angles_of_points):
                    first_list_points_in_one_line.append(white_point)


        second_list_points_in_one_line = list()
        for white_point in positions_of_white_pixels:
            if get_line_length(middle_point, white_point) > radius/2:
                angle_of_line = get_angle_of_line((middle_point, white_point))
                if compare_2_numbers_with_range(angle_of_line, list_of_angles[1], precision_of_angles_of_points):
                    second_list_points_in_one_line.append(white_point)

#         print("=========================")
#         print("first_list_points_in_one_line: ", first_list_points_in_one_line)
#         print("second_list_points_in_one_line: ", second_list_points_in_one_line)
        
        first_furthest_point = find_furthest_point_in_list(middle_point, first_list_points_in_one_line)
        second_furthest_point = find_furthest_point_in_list(middle_point, second_list_points_in_one_line)
        if len(first_list_points_in_one_line) >= n_of_pixels and len(second_list_points_in_one_line) >= n_of_pixels:               
            if check_if_perpendicular_lines((middle_point,first_furthest_point),
                                            (middle_point,second_furthest_point),
                                           accuracy_in_degrees=12):
                return True

    return False

def find_furthest_point_in_list(point, list_of_points):
    max_dist = get_line_length(point, list_of_points[0])
    max_point = list_of_points[0]
    for p in list_of_points[1:]:
        temp_dist = get_line_length(point, p)
        if temp_dist > max_dist:
            max_dist = temp_dist
            max_point = p
    return max_point

def check_if_it_is_right_angle_using_areas_of_neighborhood(image, point, radius=5, accuracy=0.04):
    slice_of_image, middle_point = slice_image_in_radius(image, point, radius)
    
    thresh = threshold_otsu(slice_of_image)
    slice_of_image_binary = slice_of_image > thresh
    labeled = label(slice_of_image_binary)

    props = regionprops(labeled)
    white_area = props[0].area
    black_area = (radius*2+1)**2-white_area
    
    white_to_black_ratio = white_area/(white_area+black_area)
#     print("white_to_black_ratio: ", white_to_black_ratio)
    return True if compare_2_numbers_with_range(0.27, white_to_black_ratio, accuracy) else False


def get_points_with_right_angle_using_areas_of_neighborhood(image):
    possible_baseline_points = corner_peaks(corner_harris(image), min_distance=1)
    points_with_right_angle = set()
    for c in possible_baseline_points:
        if check_if_it_is_right_angle_using_areas_of_neighborhood(image=image, point=(c[0],c[1]), radius=30):
            points_with_right_angle.add((c[0],c[1]))
    
    return points_with_right_angle

In [ ]:
# v5

# sprawdza wykryte po kolei kąty w wielokącie i szuka dwóch prostych obok siebie
def rotate_scale_and_cut_image_v5(image, image_path, max_height, max_width):
    image = add_black_border_to_image(image, width_of_border=30)
#     image = threshold_image(image)
    possible_baseline_points, respectively_right_angle_or_not = get_points_with_right_angle_using_order_of_angles(image)
    baseline_indices = get_points_at_the_base(possible_baseline_points, respectively_right_angle_or_not) # todo, zwróci NONE
    
    if baseline_indices is None:
        print('Nie znaleziono podstawy dla obrazka z użyciem v5: {0}'.format(image_path))
        return rotate_scale_and_cut_image_v1(image, image_path, max_height, max_width) #todo
    
    (point1_index, point2_index) = baseline_indices
    baseline = (possible_baseline_points[point1_index], possible_baseline_points[point2_index])
    baseline = [(el[1],el[0]) for el in baseline]

    angle = get_angle_of_line(baseline)
    image = rotate_image(image, angle, baseline) 
    image = cut_image_sides(image)
    image = resize_image(image, max_height, max_width)
    image = open_close_image(image)
    return image

def get_points_with_right_angle_using_order_of_angles(image):
    possible_baseline_points = find_corners(image)
#     possible_baseline_points = list(corner_peaks(corner_harris(image), min_distance=10))
#     possible_baseline_points = [list(point) for point in possible_baseline_points]
#     possible_baseline_points = possible_baseline_points+possible_baseline_points[:1]
    respectively_right_angle_or_not = list()
    possible_baseline_points2 =  list()
    for c in possible_baseline_points:
        is_right_angle, point = check_if_it_is_right_angle_using_hough(image=image, point=(c[0],c[1]), radius=20)
        if is_right_angle:
            respectively_right_angle_or_not.append(True)
            possible_baseline_points2.append(point)
        else:
            respectively_right_angle_or_not.append(False)
            possible_baseline_points2.append(point)
#     print("possible_baseline_points: ", possible_baseline_points)
#     print("respectively_right_angle_or_not: ", respectively_right_angle_or_not)
    return possible_baseline_points, respectively_right_angle_or_not

def find_corners(image):
    contour = find_contours(image, level=10)[0]
    appr_pol = list(approximate_polygon(contour, tolerance=20)[:-1]) 
#     appr_pol = list(corner_peaks(corner_harris(image), min_distance=10))
#     appr_pol_looped = appr_pol+appr_pol[:1]
    appr_pol = np.around(np.array(appr_pol))
    appr_pol = corner_subpix(image, appr_pol, window_size=20)
    appr_pol = appr_pol[~np.isnan(appr_pol).any(axis=1)]
    appr_pol = list(appr_pol)
    appr_pol = appr_pol+appr_pol[:1]
    return appr_pol

def check_if_it_is_right_angle_using_hough(image, point, radius=10):
    slice_of_image, _ = slice_image_in_radius(image, point, radius)
    list_of_angles = get_list_of_different_angles_in_image_using_hough(image=slice_of_image, radius=10)
    points = corner_peaks(corner_harris(slice_of_image), min_distance=10)
    if len(points) > 0:
        point = points[0]

#     print("point: ", point)
#     print("point: ", point)
#     print("list_of_angles: ", list_of_angles)
#     print("type list_of_angles: ", type(list_of_angles))
#     print("len list_of_angles: ", len(list_of_angles)) 
    if len(list_of_angles) == 2 and (compare_2_numbers_with_range(90, abs(list_of_angles[0]-list_of_angles[1]), range=10) or compare_2_numbers_with_range(270, abs(list_of_angles[0]-list_of_angles[1]), range=10)) :
#         print("TRUE")
#         print("=================")
        return True, point
    else:
#         print("FALSE")
#         print("=================")
        return False, point

def get_list_of_different_angles_in_image_using_hough(image, radius):
    edges = canny(image)
    lines = probabilistic_hough_line(edges, 
                                     threshold=10, 
                                     line_length=10,
                                     line_gap=1)
    list_of_angles = [get_angle_of_line(line) for line in lines]
    list_of_angles_no_duplicates = list()
#     print(len(lines))
    
    for angle in list_of_angles:
        add = True
        for added_angle in list_of_angles_no_duplicates:
            if compare_2_numbers_with_range(added_angle, angle, 5):
                add = False
        if add:
            list_of_angles_no_duplicates.append(angle)
                
    # np. kąty -176 i 180 są obok siebie, zależą od zwrotu, to samo z kątem 90
    # tutaj usuwamy takie przypadki, żeby mieć na penwo różne kąty
    exists_90 = False
    exists_180 = False
    modified_list_of_angles = list()
    for angle in list_of_angles_no_duplicates:
        if compare_2_numbers_with_range(90, abs(angle), 10):
            if exists_90 == False:
                exists_90 = True
                modified_list_of_angles.append(angle)
        elif compare_2_numbers_with_range(180, abs(angle), 10):
            if exists_180 == False:
                exists_180 = True
                modified_list_of_angles.append(angle)
        else:
            modified_list_of_angles.append(angle)
    
    return modified_list_of_angles

def get_points_at_the_base(list_of_points, respectively_right_angle_or_not):
    last_angle = None
    for i, angle in enumerate(respectively_right_angle_or_not):
        if last_angle == True and angle == True:
            return (i-1, i)
        last_angle = angle
    return None

In [ ]:
# do wersji v3 - szukanie punktów skrajnych
# def get_extreme_points(image):
#     possible_baseline_points = corner_peaks(corner_harris(image), min_distance=1)
#     minHeight = image.shape[0]-1
#     maxHeight = 0 
#     minWidth = image.shape[1]-1 
#     maxWidth = 0 
#     for height, width in possible_baseline_points:
#         if height <= minHeight:
#             minHeight = height
#         if width <= minWidth:
#             minWidth = width
            
#         if height >= maxHeight:
#             maxHeight = height
#         if width >= maxWidth:
#             maxWidth = width
            
#     extreme_points = set()
#     image = color.grey2rgb(image)
#     for height, width in detected_points:  
#         if height == minHeight or height == maxHeight or width == minWidth or width == maxWidth:
#             rr, cc = circle_perimeter(int(height),int(width), 3)
#             image[rr, cc] = [255,0,0]
#             extreme_points.add((height, width))
        
#     return image, extreme_points

### TESTOWE NARZĘDZIA ###

In [ ]:
# pełne testy wszystkich zbiorów
from pathlib import Path
n_of_sets = 7
for n in range(n_of_sets):
    set_source_path = Path("./proj1_daneA/set"+ str(n))
    set_result_path = Path("./proj1_daneA_wyniki/set"+ str(n))
    Path(set_result_path).mkdir(parents=True, exist_ok=True)

    images_paths = glob.glob(os.path.join(set_source_path, '*.png'))
    images = [io.imread(image_path) for image_path in images_paths]
    min_height, min_width = min_height_and_width_from_images(images)
    processed_images = list()
    for i, image in enumerate(images):
        _, filename = os.path.split(images_paths[i])
        
        processed_image = rotate_scale_and_cut_image_v1(image, images_paths[i], min_height, min_width)

        save_path = os.path.join(os.path.join(set_result_path, filename))
        io.imsave(save_path, arr=img_as_ubyte(processed_image))


n_of_sets = 2
for n in range(7, n_of_sets+7):
    set_source_path = Path("./proj1_daneB/set"+ str(n))
    set_result_path = Path("./proj1_daneB_wyniki/set"+ str(n))
    Path(set_result_path).mkdir(parents=True, exist_ok=True)

    images_paths = glob.glob(os.path.join(set_source_path, '*.png'))
    images = [io.imread(image_path) for image_path in images_paths]
    min_height, min_width = min_height_and_width_from_images(images)
    processed_images = list()
    for i, image in enumerate(images):
        _, filename = os.path.split(images_paths[i])
        
        processed_image = rotate_scale_and_cut_image_v1(image, images_paths[i], min_height, min_width)

        save_path = os.path.join(os.path.join(set_result_path, filename))
        io.imsave(save_path, arr=img_as_ubyte(processed_image))


# fig = plt.figure(figsize=(50,20))
# columns = 5
# rows = (len(test_set)/columns)+1
# for i, img in enumerate(processed_images):
#     fig.add_subplot(rows, columns, i+1)
#     plt.imshow(img, cmap=plt.cm.gray)


In [ ]:
# sprawdzanie wycinku w kącie
# path = os.path.join('D:\Dokumenty\Projekty\PIRO/project1/proj1_daneA\set1', '')
# images_paths = glob.glob(os.path.join(path, '*.png'))
# images = [io.imread(image_path) for image_path in images_paths]

img = io.imread("D:\\Dokumenty\\Projekty\\PIRO\\project1\\proj1_daneA\\set1\\10.png")
img = add_black_border_to_image(img)
image = threshold_image(img)

odp, p, slice = check_if_it_is_right_angle_using_hough(image, (176.45330624,  51.46785412), radius=30)
print(corner_peaks(corner_harris(slice), min_distance=10))
io.imshow((canny(slice)))

In [ ]:
# test jednego obrazka
path = os.path.join('D:\Dokumenty\Projekty\PIRO/project1/proj1_daneA\set0', '')
images_paths = glob.glob(os.path.join(path, '*.png'))
images = [io.imread(image_path) for image_path in images_paths]

max_height, max_width = min_height_and_width_from_images(images)
img_org = io.imread("D:\\Dokumenty\\Projekty\\PIRO\\project1\\proj1_daneA\\set1\\1.png")
# img_org = add_black_border_to_image(img_org, width_of_border=30)
img = rotate_scale_and_cut_image_v6(img_org, images_paths[1], max_height, max_width)
# print(corner_peaks(corner_harris(img), min_distance=1))
# img = color.gray2rgb(img)
# for angle in appr_pol:
#     rr, cc = circle(angle[0],angle[1], 3)
#     img[rr, cc] = [255,0,0]
# img[64, 69] = (255,0,0)
# img[269, 163] = (255,0,0)
img_org = add_black_border_to_image(img_org, width_of_border=30)
img_org = color.gray2rgb(img_org)
img_org[113, 55] = (255,0,0)
io.imshow(img)
#1/10 źle, wykrywa 2 kąty obok siebie w rogu

In [ ]:
io.imshow(img_org)

In [ ]:
io.imshow(rotate(img_org, 0))

In [ ]:
# wykrywanie linii
# image = io.imread(images_paths[106])
image = io.imread("D:\\Dokumenty\\Projekty\\PIRO\\project1\\proj1_daneB\\set8\\3.png")
image = color.rgb2gray(im)
edges = canny(image)
lines = probabilistic_hough_line(edges, threshold=10, line_length=10, line_gap=1)
# longest_line = (lines[0],  get_line_length(lines[0][0], lines[0][1]))
# for line in lines:
#     temp_lenght = get_line_length(line[0], line[1])
#     if temp_lenght > longest_line[1]:
#         longest_line = (line, temp_lenght)
# Generating figure 2
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(image, cmap=cm.gray)
ax[0].set_title('Input image')

ax[1].imshow(edges, cmap=cm.gray)
ax[1].set_title('Canny edges')


ax[2].imshow(edges * 0)
for line in lines:
    p0, p1 = line
    ax[2].plot((p0[0], p1[0]), (p0[1], p1[1]))
ax[2].set_xlim((0, image.shape[1]))
ax[2].set_ylim((image.shape[0], 0))
ax[2].set_title('Probabilistic Hough')

for a in ax:
    a.set_axis_off()

plt.tight_layout()
plt.show()
print(lines)
print(longest_line)

In [ ]:
# v4 - chyba się nie przyda
# wersja wycina kawałki na około wykrytych punktów i na tej podstawie szuka katów prostych i podstawy
def rotate_scale_and_cut_image_v4(image, image_path, max_height, max_width):
    points_using_angles_of_lines = get_points_with_right_angle_using_angles_of_lines(image)
    points_using_areas_of_neighborhood = get_points_with_right_angle_using_areas_of_neighborhood(image)

    baseline = get_baseline_using_corners_order(image, image_path)
    if baseline is not None:
        baseline = [(el[1],el[0]) for el in baseline] 
    else:
        print("Nie znaleziono podstawy używając kolejności kątów dla obrazu: {0}".format(image_path))
        baseline = list(points_using_angles_of_lines.intersection(points_using_areas_of_neighborhood))
        baseline = [(el[1],el[0]) for el in baseline]  
    if len(baseline) != 2:
        baseline = None
    if baseline == None:
        print('Nie znaleziono podstawy dla obrazka z użyciem v3: {0}'.format(image_path))
        return rotate_scale_and_cut_image_v1(image, image_path, max_height, max_width) #todo
    
    angle = get_angle_of_line(baseline)
    image = rotate_image(image, angle, baseline)
    image = cut_image_sides(image)
    image = resize_image(image, max_height, max_width)
    image = open_close_image(image)
    return image

def get_baseline_using_corners_order(image, image_path):
    contour = find_contours(image, level=10)[0]
    appr_pol = list(approximate_polygon(contour, tolerance=5)[:-1])    
    appr_pol = np.around(np.array(appr_pol))
    appr_pol = corner_subpix(image, appr_pol, window_size=20)
    appr_pol = appr_pol[~np.isnan(appr_pol).any(axis=1)]
    appr_pol = list(appr_pol)
    appr_pol_looped = appr_pol+appr_pol[:2]
    angles = list()
    for p1,p2,p3 in zip(appr_pol_looped[:-2], appr_pol_looped[1:-1], appr_pol_looped[2:]):
        line1 = (p1, p2)
        line2 = (p2, p3)
#         print("=======")
#         print(line1)
#         print(line2)
        if check_if_perpendicular_lines(line1, line2, accuracy_in_degrees=5):
            angles.append(np.array([p1, p2, p3]))
#     print("angles")
#     print(np.array(angles))
    right_angles = list()
    for angle in angles:
        if check_if_it_is_right_angle_using_areas_of_neighborhood(image, angle[1], radius=30, accuracy=0.04):
            right_angles.append(angle)
#     print("right_angles")
#     print(np.array(right_angles))
    if len(right_angles) == 2:
        right_angles_in_baseline = right_angles
        baseline = (right_angles_in_baseline[0][1], right_angles_in_baseline[1][1])
        return baseline
    elif len(right_angles) > 2:
        right_angles.append(right_angles[0])
        right_angles_in_baseline = list()
        for p1, p2 in zip(right_angles[:-1], right_angles[1:]):
            for p11, p22 in zip(appr_pol_looped[:-1], appr_pol_looped[1:]):
                if p1[1][0] == p11[0] and p1[1][1] == p11[1] and p2[1][0] == p22[0] and p2[1][1] == p22[1]:
                    right_angles_in_baseline.append(p1)
                    right_angles_in_baseline.append(p2)
        
        if len(right_angles_in_baseline) == 2:
            baseline = (right_angles_in_baseline[0][1], right_angles_in_baseline[1][1])
            return baseline
        else:
#             print("Nie znaleziono podstawy używając kolejności kątów dla obrazu: {0}".format(image_path))
            return None
    return None

In [ ]:
# szukanie kątów
im = io.imread("D:\\Dokumenty\\Projekty\\PIRO\\project1\\proj1_daneA\\set1\\10.png")
im = add_black_border_to_image(im, 30)
appr_pol = find_corners(im)
print(np.array(appr_pol))
im = color.gray2rgb(im)
for angle in appr_pol:
    rr, cc = circle(angle[0],angle[1], 3)
    im[rr, cc] = [255,0,0]

io.imshow(im)

In [ ]:
path = os.path.join('D:\Dokumenty\Projekty\PIRO/project1/proj1_daneA\set0', '')
images_paths = glob.glob(os.path.join(path, '*.png'))
images = [io.imread(image_path) for image_path in images_paths]
images = images
fig = plt.figure(figsize=(50,50))
columns = 5
rows = (len(images)/columns)+1
for i,im in enumerate(images):
    
    contour = find_contours(im, level=10)[0]
    appr_pol = find_corners(im)
    im = color.gray2rgb(im)
#     print(len(appr_pol))
    for angle in appr_pol:
    #     print(angle)
        rr, cc = circle(angle[0],angle[1], 3)
        im[rr, cc] = [255,0,0]
        
    fig.add_subplot(rows, columns, i+1)
    plt.imshow(im, cmap=plt.cm.gray)
    

In [ ]:
image = io.imread("D:\\Dokumenty\\Projekty\\PIRO\\project1\\proj1_daneB\\set8\\3.png")
get_angle_to_rotate_image_based_on_longest_line(image)

In [ ]:


# img = io.imread("D:\\Dokumenty\\Projekty\\PIRO\\project1\\proj1_daneA\\set1\\14.png")
# f, edges = check_if_it_is_right_angle_using_hough(img, (149, 245), radius=20)
# # im = color.rgb2gray(im)
# # slice, _ = slice_image_in_radius(im, (315.75212263, 107.31450833), 20)
# # angles = get_list_of_different_angles_in_image_using_hough(image=im, radius=20)
# # print(angles)
# io.imshow(img)

In [ ]:
# v6
# sprawdza wykryte po kolei kąty w wielokącie i szuka dwóch prostych obok siebie
def rotate_scale_and_cut_image_v6(image, image_path, max_height, max_width):
    image = add_black_border_to_image(image, width_of_border=30)
    possible_baseline_points, angles_of_points = get_points_and_angles_in_image(image)
    baseline = get_points_at_the_base(possible_baseline_points, angles_of_points) # todo, zwróci NONE
    
    if baseline is None:
        print('Nie znaleziono podstawy dla obrazka z użyciem v6: {0}'.format(image_path))
        return rotate_scale_and_cut_image_v1(image, image_path, max_height, max_width) #todo

    baseline = [(el[1],el[0]) for el in baseline]

    angle = get_angle_of_line(baseline)
    image = rotate_image(image, angle, baseline) 
    image = cut_image_sides(image)
    image = resize_image(image, max_height, max_width)
    image = open_close_image(image)
    return image

def get_points_and_angles_in_image(image):
    possible_baseline_points = find_corners(image)

    angles_of_points = list()
    for point in possible_baseline_points:
        angle = get_angle_of_point_using_hough(image, point)
        angles_of_points.append(angle)

    return possible_baseline_points, angles_of_points

def find_corners(image):
    contour = find_contours(image, level=10)[0]
    appr_pol = list(approximate_polygon(contour, tolerance=10)[:-1]) 
#     appr_pol = list(corner_peaks(corner_harris(image), min_distance=10))
#     appr_pol_looped = appr_pol+appr_pol[:1]
    appr_pol = np.around(np.array(appr_pol))
    appr_pol = corner_subpix(image, appr_pol, window_size=20)
    appr_pol = appr_pol[~np.isnan(appr_pol).any(axis=1)]
    appr_pol = list(appr_pol)
    appr_pol = appr_pol+appr_pol[:1]
    return appr_pol

def get_angle_of_point_using_hough(image, point):
    slice_of_image, _ = slice_image_in_radius(image, point, radius=20)
    list_of_angles = get_list_of_different_angles_in_image_using_hough(image=slice_of_image)
#     points = corner_peaks(corner_harris(slice_of_image), min_distance=10)
#     if len(points) > 0:
#         point = points[0]
    if len(list_of_angles) == 2:
        angle_between_lines = abs(list_of_angles[0]-list_of_angles[1])
        return  angle_between_lines
    
    return None

def get_list_of_different_angles_in_image_using_hough(image):
    edges = canny(image)
    lines = probabilistic_hough_line(edges, 
                                     threshold=10, 
                                     line_length=10,
                                     line_gap=1)
    list_of_angles = [get_angle_of_line(line) for line in lines]
    list_of_angles_no_duplicates = list()
#     print(len(lines))
    
    for angle in list_of_angles:
        add = True
        for added_angle in list_of_angles_no_duplicates:
            if compare_2_numbers_with_range(added_angle, angle, 5):
                add = False
        if add:
            list_of_angles_no_duplicates.append(angle)
                
    # np. kąty -176 i 180 są obok siebie, zależą od zwrotu, to samo z kątem 90
    # tutaj usuwamy takie przypadki, żeby mieć na penwo różne kąty
    exists_90 = False
    exists_180 = False
    modified_list_of_angles = list()
    for angle in list_of_angles_no_duplicates:
        if compare_2_numbers_with_range(90, abs(angle), 10):
            if exists_90 == False:
                exists_90 = True
                modified_list_of_angles.append(angle)
        elif compare_2_numbers_with_range(180, abs(angle), 10):
            if exists_180 == False:
                exists_180 = True
                modified_list_of_angles.append(angle)
        else:
            modified_list_of_angles.append(angle)
    
    return modified_list_of_angles

def get_points_at_the_base(list_of_points, angles_of_points):
#     print("list_of_points: ", np.array(list_of_points))
#     print("angles_of_points: ", np.array(angles_of_points))
    last_angle = None
    for i, angle in enumerate(angles_of_points):
        if last_angle is None:
            last_angle = angle
            continue
        if angle is None:
            last_angle = None
            continue
#         print((list_of_points[i-1], list_of_points[i]))
#         print(last_angle, angle)
        is_sum_180 = compare_2_numbers_with_range(last_angle + angle, 180, range=5)
        same_angles = compare_2_numbers_with_range(last_angle, angle, range=5)
        if is_sum_180 or (same_angles and angle > 70):
#             print("tak")
            return (list_of_points[i-1], list_of_points[i])
        last_angle = angle
    return None

In [ ]:
img_org = color.rgb2gray(img_org)
g, f = slice_image_in_radius(img_org, (113.27896166,  54.56640974), radius=20)
io.imshow(canny(g))

In [ ]:
get_list_of_different_angles_in_image_using_hough(g)

In [ ]:
# v7
# najpierw używa v1 a potem innego v
def rotate_scale_and_cut_image_v6(image, image_path, max_height, max_width): 
    image = threshold_image(image)
    angle, baseline = get_angle_to_rotate_image_based_on_longest_line(image)
    image = rotate_image(image, angle, baseline)
    image = cut_image_sides(image)
    image = resize_image(image, max_height, max_width)
    image = open_close_image(image)
    return image

    image = add_black_border_to_image(image, width_of_border=30)
    possible_baseline_points, angles_of_points = get_points_and_angles_in_image(image)
    baseline = get_points_at_the_base(possible_baseline_points, angles_of_points) # todo, zwróci NONE
    
    if baseline is None:
        print('Nie znaleziono podstawy dla obrazka z użyciem v6: {0}'.format(image_path))
        return rotate_scale_and_cut_image_v1(image, image_path, max_height, max_width) #todo

    baseline = [(el[1],el[0]) for el in baseline]

    angle = get_angle_of_line(baseline)
    image = rotate_image(image, angle, baseline) 
    image = cut_image_sides(image)
    image = resize_image(image, max_height, max_width)
    image = open_close_image(image)
    return image